# <center> CTA200 Computing Assignment 
<center>Daniella Morrone - daniella.morrone@mail.utoronto.ca

<center>Supervisors: Lamiya Mowla and Kartheik Iyer

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import transforms
import numpy as np
import astropy.units as u
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from photutils.aperture import aperture_photometry, CircularAperture
from numpy import unravel_index
import scipy.constants as con
from astropy.cosmology import WMAP9 as cosmo
from scipy import integrate as inte

In [ ]:
def read(filename):
    hdu = fits.open(filename)
    hdr = hdu[0].header
    data = hdu[0].data
    
    return hdr,data

# <center> Question 1 - Visualising the galaxy

##### Part 1.1
Using $\texttt{astropy.io.fits}$, open $\texttt{galaxy_hydro.fits}$ and label the different galaxy components

In [ ]:
hdr_hydro,data_hydro = read('galaxy_hydro.fits')

In [ ]:
stmass, strate, gmass, dmass = data_hydro

##### Part 1.2
Using $\texttt{matplotlib.pyplot.imshow}$ and $\texttt{matplotlib.pyplot.contour}$, plot the images of the galaxy components.

In [ ]:
log_stmass = np.log10(stmass)
log_strate = np.log10(strate)
log_gmass = np.log10(gmass)
log_dmass = np.log10(dmass)

In [ ]:
fig,axs = plt.subplots(1,4, figsize=(12,6), sharey=True, gridspec_kw={'wspace': 0.4})

def axx(i):
    axi = inset_axes(axs[i],width="3%",height="50%",loc='upper right',bbox_to_anchor=(0.06, 0., 1, 1),
                     bbox_transform=axs[i].transAxes,borderpad=0)
    return axi

# plot the image of the stellar mass
im = axs[0].imshow(stmass, cmap='pink')
axs[0].set_title('Stellar Mass', fontsize=20,pad=13)

cb = fig.colorbar(im, cax=axx(0), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=4)
cb.ax.tick_params(labelsize=12)
cb.ax.get_xaxis().labelpad = 10
cb.ax.set_xlabel('$M_\odot$', rotation=0,loc='left',fontsize=14)

# plot the image of the star formation rate
im = axs[1].imshow(strate, cmap='pink')
axs[1].set_title('Rate of \nStar Formation', fontsize=20,pad=10)

cb = fig.colorbar(im, cax=axx(1), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=4)
cb.ax.tick_params(labelsize=12)
cb.ax.get_xaxis().labelpad = 10
cb.ax.set_xlabel(r'$\dfrac{M_\odot}{yr}$', rotation=0,loc='left',fontsize=13)

# plot the image of the gas mass
im = axs[2].imshow(gmass, cmap='pink')
axs[2].set_title('Gas Mass', fontsize=20,pad=13)

cb = fig.colorbar(im, cax=axx(2), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=4)
cb.ax.tick_params(labelsize=12)
cb.ax.get_xaxis().labelpad = 10
cb.ax.set_xlabel('$M_\odot$', rotation=0,loc='left',fontsize=14)

# plot the image of the dust mass
im = axs[3].imshow(dmass, cmap='pink')
axs[3].set_title('Dust Mass', fontsize=20,pad=13)

cb = fig.colorbar(im, cax=axx(3), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=4)
cb.ax.tick_params(labelsize=12)
cb.ax.get_xaxis().labelpad = 10
cb.ax.set_xlabel('$M_\odot$', rotation=0,loc='left',fontsize=14)

for ax in axs:
    ax.set_ylim(145,40)
    ax.set_xlim(70,122)
    ax.tick_params(labelsize=14)
    ax.set_xlabel('X [pix]',fontsize=17)

axs[0].set_ylabel('Y [pix]',fontsize=17)

plt.savefig('all_mass_images.pdf')
plt.close()

In [ ]:
# plot the contours of all components on one set of axes
fig,axs = plt.subplots(1,1, figsize=(4,8))
im = axs.imshow(np.log10(stmass+gmass+dmass), cmap='twilight_shifted')

cb = fig.colorbar(im, ax=axs, orientation='vertical', aspect=50, shrink=0.65)
cb.ax.tick_params(labelsize=12)
cb.ax.get_yaxis().labelpad = 20
cb.ax.set_ylabel(r'log( $M_\odot$)',rotation=270,loc='center',fontsize=14)

#axs.contour(log_strate, cmap='winter')
#axs.contour(log_gmass, cmap='winter')
#axs.contour(log_dmass, cmap='winter')
axs.set_xlim(70,122)
axs.set_ylim(145,40)
axs.set_xlabel('X [pix]',fontsize=17)
axs.set_ylabel('Y [pix]',fontsize=17)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('All Mass Components', fontsize=20,pad=10)
plt.savefig('all_components_mass.pdf')
plt.close()

In [ ]:
# plot the contours of all components on one set of axes
fig,axs = plt.subplots(1,1, figsize=(4,8))
axs.contour(log_stmass, cmap='winter')
axs.contour(log_strate, cmap='winter')
axs.contour(log_gmass, cmap='winter')
axs.contour(log_dmass, cmap='winter')
axs.set_xlim(70,122)
axs.set_ylim(145,40)
axs.set_xlabel('X [pix]',fontsize=17)
axs.set_ylabel('Y [pix]',fontsize=17)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('All Galaxy Components', fontsize=20,pad=10)
plt.savefig('all_components_contour.pdf')
plt.close()

In [ ]:
fig,axs = plt.subplots(1,4, figsize=(12,6), sharey=True, gridspec_kw={'wspace': 0.1})

# plot the contour of the stellar mass
im = axs[0].contour(log_stmass, cmap='winter')
axs[0].set_title('Stellar Mass', fontsize=20,pad=10)

# plot the contour of the star formation rate
im = axs[1].contour(log_strate, cmap='winter')
axs[1].set_title('Rate of \nStar Formation', fontsize=20,pad=10)

# plot the contour of the gas mass
im = axs[2].contour(log_gmass, cmap='winter')
axs[2].set_title('Gas Mass', fontsize=20,pad=10)

# plot the contour of the dust mass
im = axs[3].contour(log_dmass, cmap='winter')
axs[3].set_title('Dust Mass', fontsize=20,pad=10)

for ax in axs:
    ax.set_ylim(145,40)
    ax.set_xlim(70,122)
    ax.tick_params(labelsize=14)
    ax.set_xlabel('X [pix]',fontsize=17)

axs[0].set_ylabel('Y [pix]',fontsize=17)

plt.savefig('separated_components_contour.pdf')
plt.close()

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(12,6), sharey=True, gridspec_kw={'wspace': 0.07})

# plot the image of the stellar mass with the dust mass contour overlayed
im1 = axs[0].imshow(log_stmass, cmap='twilight_shifted')
axs[0].contour(log_dmass, cmap='Greys')
axs[0].set_title('Stellar Mass \nand Dust Mass', fontsize=20)

cb = fig.colorbar(im1, ax=axs[0], orientation='vertical', aspect=50, shrink=0.65)
cb.ax.tick_params(labelsize=12)
cb.ax.get_yaxis().labelpad = 20
cb.ax.set_ylabel(r'log( $M_\odot$)',rotation=270,loc='center',fontsize=14)

# plot the image of the star formation rate with the gas mass contour overlayed
im2 = axs[1].imshow(log_strate, cmap='twilight_shifted')
axs[1].contour(log_gmass, cmap='ocean')
axs[1].set_title('Star Formation Rate \nand Gas Mass', fontsize=20)

cb = fig.colorbar(im2, ax=axs[1], orientation='vertical', aspect=55, shrink=0.65)
cb.ax.tick_params(labelsize=12)
cb.ax.get_yaxis().labelpad = 20
cb.ax.set_ylabel(r'log( $M_\odot yr^{-1}$ )',rotation=270,loc='center',fontsize=14)


for ax in axs:
    ax.tick_params(labelsize=14)
    ax.set_xlabel('X [pix]',fontsize=17)
axs[0].set_ylabel('Y [pix]',fontsize=17)
plt.savefig('contour_image_compare.pdf')
plt.close()

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(8,8), sharey=True, gridspec_kw={'wspace': 0.22})

# plot the cropped image of the stellar mass with the dust mass contour overlayed
im1 = axs[0].imshow(log_stmass, cmap='twilight_shifted')
axs[0].contour(log_dmass, cmap='Greys')
axs[0].set_title('Stellar Mass \nand Dust Mass', fontsize=20)

cb = fig.colorbar(im1, ax=axs[0], orientation='vertical', aspect=50, shrink=0.65)
cb.ax.tick_params(labelsize=12)
cb.ax.get_yaxis().labelpad = 20
cb.ax.set_ylabel(r'log( $M_\odot$)',rotation=270,loc='center',fontsize=14)

# plot the cropped image of the star formation rate with the gas mass contour overlayed
im2 = axs[1].imshow(log_strate, cmap='twilight_shifted')
axs[1].contour(log_gmass, cmap='ocean')
axs[1].set_title('Star Formation Rate \nand Gas Mass', fontsize=20)

cb = fig.colorbar(im2, ax=axs[1], orientation='vertical', aspect=55, shrink=0.65)
cb.ax.tick_params(labelsize=12)
cb.ax.get_yaxis().labelpad = 20
cb.ax.set_ylabel(r'log( $M_\odot yr^{-1}$ )',rotation=270,loc='center',fontsize=14)


for ax in axs:
    ax.set_ylim(145,40)
    ax.set_xlim(70,122)
    ax.tick_params(labelsize=14)
    ax.set_xlabel('X [pix]',fontsize=17)
axs[0].set_ylabel('Y [pix]',fontsize=17)
plt.savefig('contour_image_compare_cropped.pdf')
plt.close()

##### Part 1.3
Calculate the total stellar mass, dust mass, gas mass and star formation rate of this galaxy.

In [ ]:
# calculate the stellar mass and ensure it is consistent with the header
Stellar_Mass = np.sum(stmass)*u.M_sun
if np.round(np.log10(Stellar_Mass.value),3) == np.round(hdr_hydro['LMSTAR'],3):
    SM = Stellar_Mass    
SM

In [ ]:
# calculate the dust mass and ensure it is consistent with the header
Dust_Mass = np.sum(dmass)*u.M_sun
if np.round(np.log10(Dust_Mass.value),3) == np.round(hdr_hydro['LMDUST'],3):
    DM = Dust_Mass    
DM

In [ ]:
# calculate the gas mass and ensure it is consistent with the header
Gas_Mass = np.sum(gmass)*u.M_sun
if np.round(np.log10(Gas_Mass.value),3) == np.round(hdr_hydro['LMGAS'],3):
    GM = Gas_Mass    
GM

In [ ]:
# calculate the star formation rate and ensure it is consistent with the header
Star_Formation = np.sum(strate)*u.M_sun/u.yr
if np.round(Star_Formation.value,1) == np.round(hdr_hydro['SFR'],1):
    SF = Star_Formation    
SF

##### Part 1.1 - 2
Plot a few images of the galaxy at different wavelengths.

In [ ]:
hdr_allwav,data_allwav = read('galaxy_allwav.fits')

In [ ]:
# put all data and wavelengths into lists sorted from lowest to highest wavelength
intex = []
all_wavelengths = np.empty(data_allwav[0].shape[0])
for i in range(data_allwav[0].shape[0]):
    all_wavelengths[i] = hdr_allwav['IWAV'+str(i)]
    
sort_waves = np.sort(all_wavelengths)
index = []
for i in sort_waves:
    index.append(np.where(all_wavelengths == i)[0][0])
    
waves = np.empty(data_allwav[0].shape[0])
datas = []
for i,ind in enumerate(index):
    waves[i]=hdr_allwav['IWAV'+str(ind)]
    datas.append(data_allwav[0][ind])

In [ ]:
# define all the chosen wavelengths and data sets using variables
UVw,BLw,RDw,IRw,FIRw = hdr_allwav['IWAV0'],hdr_allwav['IWAV3'],hdr_allwav['IWAV1'],hdr_allwav['IWAV9'],hdr_allwav['IWAV17']
UV,BL,RD,IR,FIR = data_allwav[0][0], data_allwav[0][3], data_allwav[0][1], data_allwav[0][9], data_allwav[0][17]

In [ ]:
def colorplot(wavelength):
    '''
    PARAMETERS:
    wavelength <ndarray>: data set at desired wavelength
    
    RETURNS:
    image of the data with x and y densities
    '''
    
    x = np.sum(wavelength,axis=0)
    y = np.sum(wavelength,axis=1)
    
    if (wavelength == UV).all():
        title = 'Ultraviolet'
        i=0
    elif (wavelength == BL).all():
        title = 'Blue'
        i=3
    elif (wavelength == RD).all():
        title = 'Red'
        i=1
    elif (wavelength == IR).all():
        title = 'Infrared'
        i=9
    elif (wavelength == FIR).all():
        title = 'Far Infrared'
        i=17

        
    fig,axs = plt.subplots(2,2,figsize=(7,7), gridspec_kw={'wspace': 0,'hspace': 0,'height_ratios': [1, 5], 'width_ratios': [5,1]})
    axs[0,0].plot(x/max(x),c='black')
    axs[0,0].axis("off")
    axs[0,0].set_title(title, fontsize=20)

    axs[0,0].tick_params(labelsize=0,left=False)
    axs[1,0].imshow(wavelength,cmap='pink')


    base = plt.gca().transData
    rot = transforms.Affine2D().rotate_deg(270)

    axs[1,1].plot(y/max(y),c='black', transform= rot + base)
    axs[1,1].tick_params(labelsize=0,bottom=False)
    axs[1,1].axis("off")

    axs[0,1].tick_params(labelsize=0,bottom=False,left=False)
    axs[0,1].axis("off")

    axs[1,0].tick_params(labelsize=14)
    axs[1,0].set_xlabel('X [pix]',fontsize=17)
    axs[1,0].set_ylabel('Y [pix]',fontsize=17)
    
    plt.savefig('IWAV'+str(i)+'_image_dense.pdf')
    plt.close()
    
    return

In [ ]:
# plot the full images for all the chosen wavelengths as well as their x and y densities
colorplot(UV),colorplot(BL),colorplot(RD),colorplot(IR),colorplot(FIR)

In [ ]:
fig,axs = plt.subplots(1,5, figsize=(17,7), sharey=True, gridspec_kw={'wspace': 0.25})

def axx(i):
    axi = inset_axes(axs[i],width="3%",height="50%",loc='upper right',bbox_to_anchor=(0.05, 0., 1, 1),
                     bbox_transform=axs[i].transAxes,borderpad=0)
    return axi

# plot the cropped Ultraviolet image
im = axs[0].imshow(UV, cmap='pink')
axs[0].set_title('Ultraviolet \n'+str(UVw)+'µm', fontsize=20,pad=10)

cb = fig.colorbar(im, cax=axx(0), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=3)
cb.ax.tick_params(labelsize=12)

# plot the cropped Blue image
im = axs[1].imshow(BL, cmap='pink')
axs[1].set_title('Blue \n'+str(BLw)+'µm', fontsize=20,pad=10)

cb = fig.colorbar(im, cax=axx(1), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=3)
cb.ax.tick_params(labelsize=12)

# plot the cropped Red image
im = axs[2].imshow(RD, cmap='pink')
axs[2].set_title('Red \n'+str(RDw)+'µm', fontsize=20,pad=10)

cb = fig.colorbar(im, cax=axx(2), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=3)
cb.ax.tick_params(labelsize=12)

# plot the cropped Infrared image
im = axs[3].imshow(IR, cmap='pink')
axs[3].set_title('Infrared \n'+str(IRw)+'µm', fontsize=20,pad=10)

cb = fig.colorbar(im, cax=axx(3), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=3)
cb.ax.tick_params(labelsize=12)

# plot the cropped Far Infrared image
im = axs[4].imshow(FIR, cmap='pink')
axs[4].set_title('Far Infrared \n'+str(FIRw)+'µm', fontsize=20,pad=10)

cb = fig.colorbar(im, cax=axx(4), orientation='vertical', aspect=50, shrink=0.7)
cb.ax.locator_params(nbins=3)
cb.ax.tick_params(labelsize=12)

for ax in axs:
    ax.set_ylim(270,70)
    ax.set_xlim(130,250)
    ax.tick_params(labelsize=14)
    ax.set_xlabel('X [pix]',fontsize=17)

axs[0].set_ylabel('Y [pix]',fontsize=17)

plt.savefig('all_IWAV_images.pdf')
plt.close()

##### Part 1.2 - 2
Plot the total fluxes of the galaxy as a function of wavelength. 

- What does this plot tell you about how much light is emitted at different wavelengths?
- What do you think is determining how much light is emitted at different wavelengths? 
    - Younger stars emit light at shorter wavelengths (because they are hotter).
    - Dust preferentially obscures light at shorter wavelengths and re-emits them in the infrared..


In [ ]:
sumwave = np.empty(data_allwav[0].shape[0])

for i,co in enumerate(datas):
    sumwave[i]=np.sum(co)

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(15,5))

# plot the total flux for each wavelength

#ax.set_title('Total Fluxes vs. Wavelength',fontsize=20)
for ax in axs:
    ax.plot(waves,sumwave,'--*',ms=10, c='black')
    ax.set_xlabel('Wavelength [ µm ]', fontsize=17)
    ax.set_xscale('log')
    ax.grid(alpha=0.5)
    ax.tick_params(axis='x',labelsize=14)
    ax.tick_params(axis='y',labelsize=14)
    ax.yaxis.offsetText.set_fontsize(14)
axs[0].set_ylabel('Flux [ a.u. ]', fontsize=17)
axs[1].set_ylim(-1,3e4)
plt.savefig('flux_v_wave.pdf')
plt.close()

# <center> Question 2 - Galaxy Size

##### Part 2 - 1
Use $\texttt{aperture_photometry}$ to measure the circularized half-mass size of the galaxy.

Place a circular aperture of radius 10 pixels at the center of the galaxy and measure the total mass inside the aperture. Change the aperture sizes to find the radius at which the total mass inside the aperture=half the total mass of the galaxy (from previous section). Hint: Automate this by iteratively placing hundred apertures of increasing sizes! Contact me for more hints.

In [ ]:
def half_size(data,mass=False,real=False):
    '''
    PARAMETERS:
    data <ndarray>: data set being used
    mass=False or True: whether or not half mass is being found
    
    RETURNS:
    aper[h] <float>: sum within the aperture which contains half the total sum
    position <tuple>: pixel coordinates with the highest value (the centre of the galaxy)
    size_h <float>: half size in pixels
    '''
    
    # find the sum over all the pixels
    tot = np.sum(data)
    
    # find the coordinates for the center of the galaxy
    if mass == True:
        position = (96,104) # this was manually selected
    elif real == True:
        position = unravel_index(data.argmax(), data.shape) # if the real galaxy image is used
    else:
        q=np.empty(data_allwav[0].shape[0])
        y=np.empty(data_allwav[0].shape[0])
        
        # find all the coordinates with max value at different wavelengths
        for i,d in enumerate(datas):
            pos = unravel_index(d.argmax(), d.shape)
            q[i]=pos[0]
            y[i]=pos[1]
        # take median of the coordinates with max value
        position = np.median(q),np.median(y)
    
    
    x=np.linspace(1,200,1000)
    
    aper = np.empty(x.shape)
    radii = np.empty(x.shape)
    
    # iterate through different radii for the aperture photometry
    for i,rad in enumerate(x):
        aperture = CircularAperture(position, r=rad)
        a = aperture_photometry(data,aperture)[0][3]
        radii[i] = rad
        aper[i] = a

    # find where the difference between the total within the aperture and the half total is minimum
    h = np.where(aper == min(aper, key=lambda z:abs(z-tot/2)))[0][0]
    
    # find the radius of the aperture at the half size
    size_h = radii[h]
    
    return aper[h],position,size_h

##### Part 2 - 2

Find the half-mass size of the galaxy in kpc.

Use $\texttt{PIXELSCALE}$ in the $\texttt{header}$.

In [ ]:
def comp_size(pixel_size,pixel_scale,redshift):
    '''
    NOTE: this function uses astropy functions to calculate the size in kpc
            - it was not used for the computation but to check that the manual computation worked
    
    PARAMETERS:
    pixel_size <float>: size in pixels
    pixel_scale <float>: how pixels scale with arcseconds on the image
    redshift <int>: redshift of the galaxy
    
    RETURNS:
    size_kpc <astropy.units.quantity.Quantity>: size in kpc
    '''
    # add units to the pixel scale
    pixel_scale = pixel_scale*u.arcsec #per pixel

    # find how kpc scales with arcmin at the given redshift
    kpc_arcmin = (cosmo.kpc_proper_per_arcmin(redshift))
    
    # finds angular size of the galaxy
    angular_size = pixel_size*pixel_scale
    
    # find the size of the galaxy in kpc
    size_kpc = (angular_size*kpc_arcmin)   
    
    return size_kpc.to(u.kpc)

In [ ]:
def size(pixel_size, pixel_scale, redshift, Omega_M, Omega_A):
    '''
    PARAMETERS:
    pixel_size <float>: size in pixels
    pixel_scale <float>: how pixels scale with arcseconds on the image
    redshift <int>: redshift of the galaxy
    Omega_M, Omega_A <floats>: current density parameters of the universe
    Ho <astropy.units.quantity.Quantity>: current Hubble parameter of the universe
    
    RETURNS:
    length <astropy.units.quantity.Quantity>: size in kpc
    '''
    # add units to speed of light
    c = con.c *u.m/u.s 

    # add units to the pixel scale
    pixel_scale = pixel_scale*u.arcsec #per pixel
    
    # finds angular size of the galaxy
    angular_size = (pixel_size*pixel_scale).decompose()/u.rad
    
    # define the scale factor as a function of redshift
    R = lambda z: 1/(1+z)
    
    # define the derivative of scale factor as a function of density parameters and scale factor
    Rdot = lambda R: (Omega_M/R + Omega_A*R**2)**(1/2)
    
    # define function to integrate
    func = lambda R: 1/(R*Rdot(R))
    integr = inte.quad(func,R(redshift),1)
    
    # find the comoving distance (Dc) and the angular size distance (Da)
    Dc = c*integr/Ho
    Da = R(redshift)*Dc
    
    # find length using angular size and Da
    length = Da*angular_size
    
    return length[0].decompose().to(u.kpc)

##### Part 2.1
Measure the half-light size of this galaxy at optical wavelength (∼500 nm).

How does this size compare to the half-mass size of the galaxy?

In [ ]:
# define constants
pixel_scale,redshift = hdr_allwav['PIXSCALE'],hdr_allwav['Z']
Omega_M, Omega_A, Ho = cosmo.Om0, 1-cosmo.Om0, cosmo.H(0)

In [ ]:
dat500nm.shape[0]/stmass.shape[0]

In [ ]:
# find the data set closest to the given optical wavelength
i = np.where(waves == min(waves, key=lambda x:abs(x-0.5)))[0][0]
dat500nm, wav500nm = datas[i], waves[i]

In [ ]:
'''Find the half-light size'''
_,hl_pos,hl_size = half_size(dat500nm)
half_light_size500nm = size(hl_size, pixel_scale, redshift, Omega_M, Omega_A)
half_light_size500nm

In [ ]:
'''Find the half-mass size'''
_,hm_pos,hm_size = half_size(stmass+gmass+dmass,mass=True)
half_mass_size500nm = size(hm_size, pixel_scale*dat500nm.shape[0]/stmass.shape[0], redshift, Omega_M, Omega_A)
half_mass_size500nm

In [ ]:
# ratio of half mass to half light
half_mass_size500nm/half_light_size500nm

In [ ]:
y,x = hl_pos
pos = (x,y)

fig,axs = plt.subplots(1,1,figsize=(6,6))

# define the circle with radius of the half light size at the centre of the galaxy
h_light = plt.Circle(pos,hl_size,color='r',ls='-.',lw=3,fill=False,label='Half-Light')

# define the circle with radius of the half mass size at the centre of the galaxy
h_mass = plt.Circle(pos,hm_size*dat500nm.shape[0]/stmass.shape[0],color='black',ls='--',lw=3,fill=False,label='Half-Mass') 

# plot the image of the galaxy at optical wavelength
im = axs.imshow(np.log10(dat500nm),cmap='twilight')

axi = inset_axes(axs,width="3%",height="100%",loc='center right',bbox_to_anchor=(0.06, 0., 1, 1),
                 bbox_transform=axs.transAxes,borderpad=0)
cb = plt.colorbar(im,cax=axi)
cb.ax.tick_params(labelsize=12)
cb.ax.get_yaxis().labelpad = 20
cb.ax.set_ylabel(r'log( $Flux$)',rotation=270,loc='center',fontsize=14)

# plot the two circles for the half sizes
axs.add_patch(h_light)
axs.add_patch(h_mass)

axs.set_ylim(310,50)
axs.set_xlim(120,265)
axs.tick_params(labelsize=14)
axs.legend(fontsize=14)
axs.set_xlabel('X [pix]',fontsize=17)
axs.set_ylabel('Y [pix]',fontsize=17)

plt.savefig('circles.pdf')
plt.close()

##### Part 2.2
Repeat this technique to measure the sizes at all wavelengths in kpc.

In [ ]:
half_light_sizes = np.empty(data_allwav[0].shape[0])
pixel_scale,redshift = hdr_allwav['PIXSCALE'],hdr_allwav['Z']

for i,dat in enumerate(datas):
    _,_,hlsize = half_size(dat)
    half_light_sizes[i] = size(hlsize, pixel_scale, redshift, Omega_M, Omega_A).value

##### Park 2.3
Plot size vs. wavelength of this galaxy. Over-plot the half-mass size of the galaxy as dashed line.

In [ ]:
plt.figure(figsize=(8,6))
# plot the half light sizes at each wavelength
plt.plot(waves,half_light_sizes,'-.o',c='black',ms=7,label='Half-Light')

# plot the half mass size
plt.axhline(y = half_mass_size500nm.value, color='indigo', linestyle='--',label='Half-Mass')

plt.grid(alpha=0.5)
plt.xscale('log')
plt.xlabel('Wavelength [µm]',fontsize=17)
plt.ylabel('Size [kpc]',fontsize=17)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(6,12)
plt.legend(fontsize=14,loc=2)
plt.savefig('half_v_wave.pdf')
plt.close()

In [ ]:
# find the ratio of half mass to half light at all wavelenghts 
ratios = half_mass_size500nm/(half_light_sizes*u.kpc)
ratios

##### Part 2.4
Here is an image on the same galaxy on the sky $\texttt{galaxy_onsky_F160W.fits}$. Can you use the methods described above to measure the size of the galaxy from this image? Explain why or why not.

In [ ]:
exponent = np.log(data_onsky) / np.log(0.01)
clean = 900**(3*exponent)

In [ ]:
datbool = np.empty(clean.shape, dtype=bool)
for i in range(194):
    for j in range(194):
        if i < 75 or i > 115 or j > 125 or j < 75:
            datbool[j,i] = False
        else:
            if clean[j,i]>100:
                datbool[j,i] = True
            elif clean[j,i]<100:
                datbool[j,i] = False

In [ ]:
x0 = np.mean(data_onsky,axis=0)
y0 = np.mean(data_onsky,axis=1)
x1 = np.mean(data_onsky*datbool,axis=0)
y1 = np.mean(data_onsky*datbool,axis=1)

fig,axs = plt.subplots(1,2, sharey=True, figsize=(14,4), gridspec_kw={'wspace': 0.1})
axs[0].plot(x0/max(x0), c='black', ls='--', label='X values')
axs[0].plot(y0/max(y0), alpha=0.8, c='green', ls='-', label='Y values')
axs[0].set_title('Unfiltered', fontsize=19)

axs[1].plot(x1/max(x1), c='black', ls='--', label='X values')
axs[1].plot(y1/max(y1), alpha=0.8, c='green', ls='-', label='Y values')
axs[1].set_title('Filtered', fontsize=19)

for ax in axs:
    ax.legend(fontsize=14)
    ax.tick_params(labelsize=14)
    ax.set_xlabel('Pixel', fontsize=16)
axs[0].set_ylabel('Flux Density', fontsize=16)
plt.savefig('onsky_filt.pdf')
plt.close()

In [ ]:
data = data_onsky*datbool
_,_,size_onsky = halfsize(data)
pixelscale,reds = hdr_onsky['PIXSCALE'], hdr_onsky['Z']
size_onsky,size(size_onsky, pixelscale, reds, Omega_M, Omega_A)

# <center> Question 3 - Using $\texttt{statmorph}$

##### Part 3-1
Install $\texttt{statmorph}$ - ensure that this is installed before running the following script on your comuter

In [ ]:
import statmorph
from statmorph.utils.image_diagnostics import make_figure

In [ ]:
import scipy.ndimage as ndi
from astropy.visualization import simple_norm
from astropy.modeling import models
from astropy.convolution import convolve
import photutils
import time
%matplotlib inline

##### Part 3-2
Follow the tutorial to make sure $\texttt{statmorph}$ can run.

In [ ]:
ny, nx = 240, 240
y, x = np.mgrid[0:ny, 0:nx]
sersic_model = models.Sersic2D(
    amplitude=1, r_eff=20, n=2.5, x_0=120.5, y_0=96.5,
    ellip=0.5, theta=-0.5)
image = sersic_model(x, y)
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
plt.show()

size = 20  # on each side from the center
sigma_psf = 2.0
y, x = np.mgrid[-size:size+1, -size:size+1]
psf = np.exp(-(x**2 + y**2)/(2.0*sigma_psf**2))
psf /= np.sum(psf)
plt.imshow(psf, origin='lower', cmap='gray')
plt.show()

In [ ]:
image = convolve(image, psf)
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
plt.show()

np.random.seed(1)
snp = 100.0
image += (1.0 / snp) * np.random.standard_normal(size=(ny, nx))
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
plt.show()

In [ ]:
gain = 10000.0
threshold = photutils.detect_threshold(image, 1.5)
npixels = 5  # minimum number of connected pixels
segm = photutils.detect_sources(image, threshold, npixels)

In [ ]:
# Keep only the largest segment
label = np.argmax(segm.areas) + 1
segmap = segm.data == label
plt.imshow(segmap, origin='lower', cmap='gray')
plt.show()

segmap_float = ndi.uniform_filter(np.float64(segmap), size=10)
segmap = segmap_float > 0.5
plt.imshow(segmap, origin='lower', cmap='gray')
plt.show()

In [ ]:
start = time.time()
source_morphs = statmorph.source_morphology(
    image, segmap, gain=gain, psf=psf)
print('Time: %g s.' % (time.time() - start))

In [ ]:
morph = source_morphs[0]

In [ ]:
print('xc_centroid =', morph.xc_centroid)
print('yc_centroid =', morph.yc_centroid)
print('ellipticity_centroid =', morph.ellipticity_centroid)
print('elongation_centroid =', morph.elongation_centroid)
print('orientation_centroid =', morph.orientation_centroid)
print('xc_asymmetry =', morph.xc_asymmetry)
print('yc_asymmetry =', morph.yc_asymmetry)
print('ellipticity_asymmetry =', morph.ellipticity_asymmetry)
print('elongation_asymmetry =', morph.elongation_asymmetry)
print('orientation_asymmetry =', morph.orientation_asymmetry)
print('rpetro_circ =', morph.rpetro_circ)
print('rpetro_ellip =', morph.rpetro_ellip)
print('rhalf_circ =', morph.rhalf_circ)
print('rhalf_ellip =', morph.rhalf_ellip)
print('r20 =', morph.r20)
print('r80 =', morph.r80)
print('Gini =', morph.gini)
print('M20 =', morph.m20)
print('F(G, M20) =', morph.gini_m20_bulge)
print('S(G, M20) =', morph.gini_m20_merger)
print('sn_per_pixel =', morph.sn_per_pixel)
print('C =', morph.concentration)
print('A =', morph.asymmetry)
print('S =', morph.smoothness)
print('sersic_amplitude =', morph.sersic_amplitude)
print('sersic_rhalf =', morph.sersic_rhalf)
print('sersic_n =', morph.sersic_n)
print('sersic_xc =', morph.sersic_xc)
print('sersic_yc =', morph.sersic_yc)
print('sersic_ellip =', morph.sersic_ellip)
print('sersic_theta =', morph.sersic_theta)
print('sky_mean =', morph.sky_mean)
print('sky_median =', morph.sky_median)
print('sky_sigma =', morph.sky_sigma)
print('flag =', morph.flag)
print('flag_sersic =', morph.flag_sersic)

In [ ]:
ny, nx = image.shape
y, x = np.mgrid[0:ny, 0:nx]
fitted_model = statmorph.ConvolvedSersic2D(
    amplitude=morph.sersic_amplitude,
    r_eff=morph.sersic_rhalf,
    n=morph.sersic_n,
    x_0=morph.sersic_xc,
    y_0=morph.sersic_yc,
    ellip=morph.sersic_ellip,
    theta=morph.sersic_theta)
fitted_model.set_psf(psf)  # required when using ConvolvedSersic2D
image_model = fitted_model(x, y)
bg_noise = (1.0 / snp) * np.random.standard_normal(size=(ny, nx))
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(131)
ax.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
ax.set_title('Original image')
ax = fig.add_subplot(132)
ax.imshow(image_model + bg_noise, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
ax.set_title('Fitted model')
ax = fig.add_subplot(133)
residual = image - image_model
ax.imshow(residual, cmap='gray', origin='lower',
           norm=simple_norm(residual, stretch='linear'))
ax.set_title('Residual')

In [ ]:
fig = make_figure(morph)

In [ ]:
plt.close(fig)

##### Part 3-3
Take a shot at measuring the morphological parameters of this example galaxy $\texttt{galaxy_onsky_F160W.fits}$.

In [ ]:
hdr_onsky,data_onsky = read('galaxy_onsky_F160W.fits')

In [ ]:
image=data_onsky
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
plt.show()
size = 20  # on each side from the center
sigma_psf = 2.0
y, x = np.mgrid[-size:size+1, -size:size+1]
psf = np.exp(-(x**2 + y**2)/(2.0*sigma_psf**2))
psf /= np.sum(psf)
#plt.imshow(psf, origin='lower', cmap='gray')
image = convolve(image, psf)
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
plt.show()

In [ ]:
gain = 100.0
threshold = photutils.detect_threshold(image, 1.1)
npixels = 5  # minimum number of connected pixels
segm = photutils.detect_sources(image, threshold, npixels)

# Keep only the largest segment
label = np.argmax(segm.areas) + 1
segmap = segm.data == label
#plt.imshow(segmap, origin='lower', cmap='gray')

segmap_float = ndi.uniform_filter(np.float64(segmap), size=10)
segmap = segmap_float > 0.5
plt.imshow(segmap, origin='lower', cmap='gray')
plt.show()

In [ ]:
start = time.time()
source_morphs = statmorph.source_morphology(
    image, segmap, gain=gain, psf=psf)
print('Time: %g s.' % (time.time() - start))

In [ ]:
morph = source_morphs[0]

In [ ]:
print('xc_centroid =', morph.xc_centroid)
print('yc_centroid =', morph.yc_centroid)
print('ellipticity_centroid =', morph.ellipticity_centroid)
print('elongation_centroid =', morph.elongation_centroid)
print('orientation_centroid =', morph.orientation_centroid)
print('xc_asymmetry =', morph.xc_asymmetry)
print('yc_asymmetry =', morph.yc_asymmetry)
print('ellipticity_asymmetry =', morph.ellipticity_asymmetry)
print('elongation_asymmetry =', morph.elongation_asymmetry)
print('orientation_asymmetry =', morph.orientation_asymmetry)
print('rpetro_circ =', morph.rpetro_circ)
print('rpetro_ellip =', morph.rpetro_ellip)
print('rhalf_circ =', morph.rhalf_circ)
print('rhalf_ellip =', morph.rhalf_ellip)
print('r20 =', morph.r20)
print('r80 =', morph.r80)
print('Gini =', morph.gini)
print('M20 =', morph.m20)
print('F(G, M20) =', morph.gini_m20_bulge)
print('S(G, M20) =', morph.gini_m20_merger)
print('sn_per_pixel =', morph.sn_per_pixel)
print('C =', morph.concentration)
print('A =', morph.asymmetry)
print('S =', morph.smoothness)
print('sersic_amplitude =', morph.sersic_amplitude)
print('sersic_rhalf =', morph.sersic_rhalf)
print('sersic_n =', morph.sersic_n)
print('sersic_xc =', morph.sersic_xc)
print('sersic_yc =', morph.sersic_yc)
print('sersic_ellip =', morph.sersic_ellip)
print('sersic_theta =', morph.sersic_theta)
print('sky_mean =', morph.sky_mean)
print('sky_median =', morph.sky_median)
print('sky_sigma =', morph.sky_sigma)
print('flag =', morph.flag)
print('flag_sersic =', morph.flag_sersic)

In [ ]:
ny, nx = image.shape
y, x = np.mgrid[0:ny, 0:nx]
fitted_model = statmorph.ConvolvedSersic2D(
    amplitude=morph.sersic_amplitude,
    r_eff=morph.sersic_rhalf,
    n=morph.sersic_n,
    x_0=morph.sersic_xc,
    y_0=morph.sersic_yc,
    ellip=morph.sersic_ellip,
    theta=morph.sersic_theta)
fitted_model.set_psf(psf)  # required when using ConvolvedSersic2D
image_model = fitted_model(x, y)
bg_noise = (1.0 / snp) * np.random.standard_normal(size=(ny, nx))
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(131)
ax.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
ax.set_title('Original image')
ax = fig.add_subplot(132)
ax.imshow(image_model + bg_noise, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
ax.set_title('Fitted model')
ax = fig.add_subplot(133)
residual = image - image_model
ax.imshow(residual, cmap='gray', origin='lower',
           norm=simple_norm(residual, stretch='linear'))
ax.set_title('Residual')

plt.show()

In [ ]:
fig = make_figure(morph)
plt.savefig('statmorph.pdf')
plt.close(fig)